In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape
from tensorflow.keras.utils import to_categorical

# Load preprocessed features and labels
X = np.load('extracted_features.npy', allow_pickle=True)
y = np.load('labels.npy', allow_pickle=True)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y_cat)

# Reshape input for CNN (assume each sample is 2D: (n_features, time_steps))
X_train = X_train[..., np.newaxis]  # Shape: (num_samples, n_features, time_steps, 1)
X_test = X_test[..., np.newaxis]

print("Training shape:", X_train.shape)
print("Test shape:", X_test.shape)

In [ ]:
# CNN
cnn_model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_cat.shape[1], activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.summary()

history_cnn = cnn_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
# CNN + LSTM
X_train_seq = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])  # (samples, time, features)
X_test_seq = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])

cnn_lstm_model = Sequential([
    TimeDistributed(Dense(64), input_shape=X_train_seq.shape[1:]),
    LSTM(128, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_cat.shape[1], activation='softmax')
])

cnn_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_lstm_model.summary()

history_lstm = cnn_lstm_model.fit(X_train_seq, y_train, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
def plot_confusion(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# CNN Evaluation
y_pred_cnn = cnn_model.predict(X_test)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)
y_true = np.argmax(y_test, axis=1)

print("CNN Classification Report:")
print(classification_report(y_true, y_pred_cnn_classes, target_names=label_encoder.classes_))
plot_confusion(y_true, y_pred_cnn_classes, "CNN Confusion Matrix")

# CNN + LSTM Evaluation
y_pred_lstm = cnn_lstm_model.predict(X_test_seq)
y_pred_lstm_classes = np.argmax(y_pred_lstm, axis=1)

print("CNN + LSTM Classification Report:")
print(classification_report(y_true, y_pred_lstm_classes, target_names=label_encoder.classes_))
plot_confusion(y_true, y_pred_lstm_classes, "CNN + LSTM Confusion Matrix")